# Analyze text and perform NLP(Natural Language Processing) tasks

In [2]:
# Importing neccessary packages

import pyodbc             # Connect to SQL Server and load data
import pandas as pd
import nltk               # Analyze text and perform NLP tasks
from nltk.sentiment import SentimentIntensityAnalyzer


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Download sentiment lexicon (run once)

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MERCY\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

**This is a general function that connects to the sql server, where you can call any table**

* Create a connection to SQL Server
* **DRIVER:** the database driver to use (SQL Server)
* **SERVER:** your local machine name and SQL Server instance
* **DATABASE:** database name (passed as a function argument)
* **Trusted_Connection=yes** means it uses Windows Authentication instead of username/password

def fetch_data(query, database="Marketing_Analytics"):

    conn = pyodbc.connect(
    
        f'DRIVER={{SQL Server}};
          SERVER=DESKTOP-0G79JHF\\SQLEXPRESS;
          DATABASE={database};
          Trusted_Connection=yes;'
    )
    df = pd.read_sql(query, conn)
    conn.close()
    return df

**Now you can reuse it:**

reviews = fetch_data("SELECT * FROM dbo.customer_reviews;")

customers = fetch_data("SELECT * FROM dbo.customer_data;")

In [5]:
# Define a function that will connect to SQL Server, run a query, and return the results as a DataFrame

def fetch_data(query, database="Marketing_Analytics"):
    conn_str = (
        f'DRIVER={{SQL Server}};'
        f'SERVER=DESKTOP-0G79JHF\\SQLEXPRESS;'
        f'DATABASE={database};'
        'Trusted_Connection=yes;'
    )

    #connect and query
    conn = pyodbc.connect(conn_str)    # Connect to SQL Server using the connection string
    df = pd.read_sql(query, conn)      # Execute the query and load the result into a pandas DataFrame
    conn.close                         # Close the connection after use
    return df                          # Return the DataFrame


In [6]:
# Fetch the customer reviews from the SQL Database

customer_reviews_df = fetch_data('SELECT * FROM dbo.customer_reviews_cleaned')
customer_reviews_df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [7]:
# Creating an object that calls the analyzer known as SentimentIntensityAnalyzer()

sia = SentimentIntensityAnalyzer()

In [8]:
# Example of how the sia works

reviews = [
    "This product is terrible and broke on the first day.",
    "It’s okay, not great but not bad either.",
    "Absolutely fantastic! Highly recommend it."
]

for r in reviews:
    print(r)
    print(sia.polarity_scores(r))
    print()


This product is terrible and broke on the first day.
{'neg': 0.424, 'neu': 0.576, 'pos': 0.0, 'compound': -0.7096}

It’s okay, not great but not bad either.
{'neg': 0.174, 'neu': 0.404, 'pos': 0.422, 'compound': 0.4728}

Absolutely fantastic! Highly recommend it.
{'neg': 0.0, 'neu': 0.293, 'pos': 0.707, 'compound': 0.8043}



In [9]:
# Define a function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    # Get the sentiment scores for the review text
    sentiment = sia.polarity_scores(review)
    # Return the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)
    return sentiment['compound']

In [10]:
# Define a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment

In [14]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

In [11]:
# Apply sentiment analysis to calculate sentiment scores for each review
    # calculate_sentiment, is the function we created to give us the score
    # here we want to create a column that stores the score for each review

    # apply() is used to apply a function to each row

customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [12]:
# Apply sentiment categorization using both text and rating
    # Creating a column for the SentimentCategory

    # lamda is used to define an expression,that is, lambda x:expression, means "for each value x, return this result


# Create a new column 'SentimentCategory' by applying a custom function to each row.
# The lambda function passes both 'SentimentScore' and 'Rating' from each row to the 'categorize_sentiment' function.
# The 'axis=1' ensures that the function is applied row by row (not column by column).
# This helps combine both sentiment score and rating to assign a meaningful sentiment category.


customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

In [15]:
# Apply sentiment bucketing to categorize scores into defined ranges

customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

In [16]:
# Display the first few rows of the DataFrame with sentiment scores, categories, and buckets

print(customer_reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                             ReviewText  SentimentScore SentimentCategory  \
0  Average experience, nothing special.         -0.3089    Mixed Negative   
1            The quality is  top-notch.          0.0000          Positive   
2    Five stars for the quick delivery.          0.0000          Positive   
3   Good quality, but could be cheaper.          0.2382    Mixed Positive   
4  Average experience, nothing special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [18]:
# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file

customer_reviews_df.to_csv('customer_reviews_cleaned_with_sentiment.csv', index=False)